<a href="https://colab.research.google.com/github/dung18520632/CS114.K21/blob/master/VGG_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CÀI ĐẶT CÁC THƯ VIỆN**

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from PIL import Image
import os

# **CONNECTED MY KAGGLE**

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hoangvandung","key":"536b985b25a4e735caa0dff6dcb92a3b"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# **TẢI XUỐNG BỘ DATASET BIỂN BÁO**

In [4]:
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign --force

 99% 608M/612M [00:12<00:00, 87.1MB/s]
100% 612M/612M [00:12<00:00, 50.0MB/s]


# **GIẢI NÉN DATASET**

In [ ]:
!unzip gtsrb-german-traffic-sign.zip

# **TẢI DỮ LIỆU IMAGE VÀ LABEL**

In [ ]:
data=[]
labels=[]

for i in range(43) :
    path = "/content/Train/{0}/".format(i)
    Class=os.listdir(path)#Trả về các thư mục con từ đường dẫn đã cho
    print(path)
    for a in Class:
        try:
            image=cv2.imread(path+a)
            image_from_array = Image.fromarray(image, 'RGB')#Chuyển các bức ảnh thành xám
            size_image = image_from_array.resize((64, 64))# Các bức ảnh có các size khác nhau, ta thay đổi size của chúng về chung (64,64)
            data.append(np.array(size_image))#Thêm ảnh vào data dưới dạng ma trận
            labels.append(i)#Đánh số labels
        except AttributeError:
            print(" ")

# **KÍCH THƯỚC DATA VÀ LABELS**

In [15]:
print(len(data))
print(len(labels))

39209
39209


# **CHUYỂN DỮ LIỆU THÀNH MA TRẬN** 

In [16]:
Cells=np.array(data)
labels=np.array(labels)
#Lưu ma trận 
np.save("Cells",Cells)
np.save("labels",labels)

In [17]:
#Load ma trận
Cells=np.load("Cells.npy")
labels=np.load("labels.npy")

# **TẠO RA BỘ DỮ LIỆU NGẪU NHIÊN**

In [25]:
s=np.arange(Cells.shape[0])
np.random.seed(43)
np.random.shuffle(s)
Cells=Cells[s]
labels=labels[s]

# **CHIA DỮ LIỆU THÀNH PHẦN TRAIN VÀ PHẦN TEST**

In [29]:
(trainX,testX)=Cells[(int)(0.2*len(labels)):],Cells[:(int)(0.2*len(labels))]
trainX = trainX.astype('float32')/255 #
testX = testX.astype('float32')/255
(trainY,testY)=labels[(int)(0.2*len(labels)):],labels[:(int)(0.2*len(labels))]
#Using one hote encoding for the train and validation labels
from keras.utils import to_categorical
trainY = to_categorical(trainY, 43)
testY = to_categorical(testY, 43)
print(len(trainX))
print(len(trainY))
print(len(testX))
print(len(testY))


31368
31368
7841
7841


In [30]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras import applications
from tensorflow.keras import optimizers
model = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(trainX.shape[1:]))
x=Sequential()
x.add(Flatten(input_shape=model.output_shape[1:]))
x.add(Dense(256, activation='relu'))
x.add(Dense(43, activation='sigmoid'))
model = Model(inputs=model.input, outputs=x(model.output))
model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()


58892288/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
______________________________________________________________

In [31]:
opt = Adam(lr=1e-4)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [32]:
print("[INFO] training ...")
model.fit(trainX, trainY, validation_data=(testX, testY),
  batch_size=64, epochs=10, verbose=1)


[INFO] training ...
Epoch 1/10
491/491 [==============================] - 26s 52ms/step - loss: 0.0255 - accuracy: 0.8279 - val_loss: 0.0041 - val_accuracy: 0.9791
Epoch 2/10
491/491 [==============================] - 25s 51ms/step - loss: 0.0019 - accuracy: 0.9900 - val_loss: 0.0016 - val_accuracy: 0.9911
Epoch 3/10
491/491 [==============================] - 25s 51ms/step - loss: 7.9635e-04 - accuracy: 0.9960 - val_loss: 0.0041 - val_accuracy: 0.9784
Epoch 4/10
491/491 [==============================] - 25s 51ms/step - loss: 9.5884e-04 - accuracy: 0.9952 - val_loss: 0.0016 - val_accuracy: 0.9913
Epoch 5/10
491/491 [==============================] - 25s 51ms/step - loss: 4.8599e-04 - accuracy: 0.9977 - val_loss: 7.3830e-04 - val_accuracy: 0.9971
Epoch 6/10
491/491 [==============================] - 25s 51ms/step - loss: 4.0704e-04 - accuracy: 0.9978 - val_loss: 7.6105e-04 - val_accuracy: 0.9960
Epoch 7/10
491/491 [==============================] - 25s 51ms/step - loss: 5.8552e-04 - acc

In [ ]:
from tensorflow.keras.models import load_model

print("[INFO] save model...")
model.save('/content/vgg16.model')

[INFO] save model...
INFO:tensorflow:Assets written to: /content/vgg16.model/assets


In [ ]:
from tensorflow.keras.models import load_model

print("[INFO] loading model...")
model = load_model('/content/vgg16.model')

[INFO] loading model...


In [ ]:
#Predicting with the test data
import pandas as pd
y_test=pd.read_csv("/content/Test.csv")
y_test.head()
labels=y_test['Path'].to_numpy() 
y_test=y_test['ClassId'].values

data=[]

for f in labels:
    image_test=cv2.imread('/content/test/'+f.replace('Test/', ''))
    image_from_array_test = Image.fromarray(image_test, 'RGB')
    size_image = image_from_array_test.resize((64, 64))
    data.append(np.array(size_image))

X_test=np.array(data)
X_test = X_test.astype('float32')/255 
predictions = model.predict(X_test ,batch_size=64)
pred = predictions.argmax(axis=1)

In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, pred))

0.9719714964370546
